Worksheet 1 Introduction to linear algebra
by Prof J.Morlier February 2020

#import Base.print_matrix
#import Pkg; Pkg.add("SymPy")

In [157]:
using LinearAlgebra, SparseArrays, SuiteSparse, Random
using SymPy
import Base.print_matrix



In [156]:
versioninfo()

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


A 2x2 Recap

In [83]:
entries = @syms a b c d  real=true


(a, b, c, d)

In [84]:
M = [a b; c d]


2×2 Array{Sym,2}:
 a  b
 c  d

In [85]:
dM= det(M) 

a⋅d - b⋅c

How can I invert a 3x3 matrix ?

In [86]:
entries = @syms a b c d e f g h i real=true


(a, b, c, d, e, f, g, h, i)

In [87]:
A = reshape([entries...],(3,3))


3×3 Array{Sym,2}:
 a  d  g
 b  e  h
 c  f  i

In [88]:

detA = det(A) |> simplify

a⋅e⋅i - a⋅f⋅h - b⋅d⋅i + b⋅f⋅g + c⋅d⋅h - c⋅e⋅g

on the SVD of matrix A of size 2 x 3

In [89]:
A=[4 11 14; 8 7 -2]; show(A)

[4 11 14; 8 7 -2]

Construct $B=A^TA$

In [90]:
B=transpose(A)*A; show(B)

[80 100 40; 100 170 140; 40 140 200]

#Check;
# B=[80 100 40; 100 170 140; 40 140 200];

In [91]:
lambda = symbols("lambda", real=true)


λ

In [92]:
C=B-lambda*[1 0 0; 0 1 0; 0 0 1]

3×3 Array{Sym,2}:
 80 - lambda           100            40
         100  170 - lambda           140
          40           140  200 - lambda

In [93]:
dC=det(C)

31200⋅λ + (80 - λ)⋅(170 - λ)⋅(200 - λ) - 2720000

In [94]:
simplify(dC)

  ⎛   2                ⎞
λ⋅⎝- λ  + 450⋅λ - 32400⎠

In [95]:
eqn = -lambda^3 + 450*lambda^2 -32400*lambda

   3        2          
- λ  + 450⋅λ  - 32400⋅λ

In [96]:
real_roots(eqn)

3-element Array{Sym,1}:
   0
  90
 360

In [97]:
solve(eqn)

3-element Array{Sym,1}:
   0
  90
 360

In [98]:
factor(-lambda^3 + 450*lambda^2 -32400*lambda,lambda)

-λ⋅(λ - 360)⋅(λ - 90)

In [99]:
#recap
#A=[4 11 14; 8 7 -2];

In [100]:
A=[4 11 14; 8 7 -2];

https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.svd

In [101]:
U,S,V = svd(A)

SVD{Float64,Float64,Array{Float64,2}}
U factor:
2×2 Array{Float64,2}:
 -0.948683  -0.316228
 -0.316228   0.948683
singular values:
2-element Array{Float64,1}:
 18.973665961010283
  9.486832980505136
Vt factor:
2×3 Array{Float64,2}:
 -0.333333  -0.666667  -0.666667
  0.666667   0.333333  -0.666667

In [131]:
function fullsvd(A) 
    U,s,V = svd(A, full = true)  # compute svd
    Σ = zeros(size(A))           # container for Σ  
    for i=1:length(s)
       Σ[i,i] = s[i]            # place singular values in Σ
    end                      # a practical svd would never store all these zeros
    display(U);display(Σ);display(V) # display the answer
    return(U,Σ,V)                # return the answer
end





fullsvd (generic function with 1 method)

https://github.com/mitmath/1806/blob/master/summaries.md

In [132]:
function rankrsvd(A) 
    U,s,V = svd(A, full = true)  # compute svd
    r = sum(s.>1e-8)             # rank = how many positive?
    U₁ = U[:,1:r]
    Σᵣ = Diagonal(s[1:r])        # Diagonal matrix of singular values
    V₁ = V[:,1:r]
    display(U₁);display(Σᵣ);display(V₁) # display the answer
    return(U₁,Σᵣ,V₁)                # return the answer
end

rankrsvd (generic function with 1 method)

In [135]:
U,S,V = fullsvd(A) #matlab notation

2×2 Array{Float64,2}:
 -0.948683  -0.316228
 -0.316228   0.948683

2×3 Array{Float64,2}:
 18.9737  0.0      0.0
  0.0     9.48683  0.0

3×3 Adjoint{Float64,Array{Float64,2}}:
 -0.333333   0.666667  -0.666667
 -0.666667   0.333333   0.666667
 -0.666667  -0.666667  -0.333333

([-0.9486832980505135 -0.3162277660168378; -0.3162277660168378 0.9486832980505138], [18.973665961010283 0.0 0.0; 0.0 9.486832980505136 0.0], [-0.3333333333333332 0.6666666666666665 -0.6666666666666666; -0.6666666666666665 0.3333333333333334 0.6666666666666669; -0.6666666666666665 -0.6666666666666671 -0.3333333333333332])

In [136]:
print_matrix(stdout, U);

 -0.9486832980505135  -0.3162277660168378
 -0.3162277660168378   0.9486832980505138

In [137]:
print_matrix(stdout,S);


 18.973665961010283  0.0                0.0
  0.0                9.486832980505136  0.0

In [138]:
print_matrix(stdout, V);

 -0.3333333333333332   0.6666666666666665  -0.6666666666666666
 -0.6666666666666665   0.3333333333333334   0.6666666666666669
 -0.6666666666666665  -0.6666666666666671  -0.3333333333333332

Need to check that $(B-\lambda_i I)v_i=0$

In [139]:
#check for lambda_1
 360*V[:,1]

3-element Array{Float64,1}:
 -119.99999999999996
 -239.99999999999994
 -239.99999999999994

In [140]:
alpha=360

360

In [141]:
(B)*V[:,1]

3-element Array{Float64,1}:
 -119.99999999999997
 -239.99999999999994
 -239.99999999999994

In [142]:
#check done

Pseudo inverse and linear system

In [143]:
b=[1;2]

2-element Array{Int64,1}:
 1
 2

In [144]:
rang_A=rank(A)

2

In [145]:
#Non inversible == square matrix
# 1/A
# inv(A)
#Least square possible using pinv

pinv(A)

3×2 Array{Float64,2}:
 -0.00555556   0.0722222
  0.0222222    0.0444444
  0.0555556   -0.0555556

In [146]:
xstar=pinv(A)*b

3-element Array{Float64,1}:
  0.13888888888888884
  0.11111111111111109
 -0.0555555555555557 

In [147]:
#check 1



In [148]:
pseudoA1=V*pinv(S)*U'

3×2 Array{Float64,2}:
 -0.00555556   0.0722222
  0.0222222    0.0444444
  0.0555556   -0.0555556

In [152]:

xstar1=pseudoA1*b

3-element Array{Float64,1}:
  0.13888888888888884 
  0.11111111111111109 
 -0.055555555555555684

In [153]:
St=Sn[1:2,1:2]

2×2 Array{Float64,2}:
 18.9737  0.0    
  0.0     9.48683

In [154]:
pseudoA2=V*[inv(St'*St)*St'; 0 0 ]*U'

3×2 Array{Float64,2}:
 -0.00555556   0.0722222
  0.0222222    0.0444444
  0.0555556   -0.0555556

In [155]:
 xstar2=pseudoA2*b

3-element Array{Float64,1}:
  0.13888888888888884 
  0.11111111111111109 
 -0.055555555555555684